# Sentiment Analysis

## Using XGBoost in SageMaker

_Deep Learning Nanodegree Program | Deployment_

---

As our first example of using Amazon's SageMaker service we will construct a random tree model to predict the sentiment of a movie review. You may have seen a version of this example in a pervious lesson although it would have been done using the sklearn package. Instead, we will be using the XGBoost package as it is provided to us by Amazon.

## Instructions

Some template code has already been provided for you, and you will need to implement additional functionality to successfully complete this notebook. You will not need to modify the included code beyond what is requested. Sections that begin with '**TODO**' in the header indicate that you need to complete or implement some portion within them. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `# TODO: ...` comment. Please be sure to read the instructions carefully!

In addition to implementing code, there may be questions for you to answer which relate to the task and your implementation. Each section where you will answer a question is preceded by a '**Question:**' header. Carefully read each question and provide your answer below the '**Answer:**' header by editing the Markdown cell.

> **Note**: Code and Markdown cells can be executed using the **Shift+Enter** keyboard shortcut. In addition, a cell can be edited by typically clicking it (double-click for Markdown cells) or by pressing **Enter** while it is highlighted.

## Step 1: Downloading the data

The dataset we are going to use is very popular among researchers in Natural Language Processing, usually referred to as the [IMDb dataset](http://ai.stanford.edu/~amaas/data/sentiment/). It consists of movie reviews from the website [imdb.com](http://www.imdb.com/), each labeled as either '**pos**itive', if the reviewer enjoyed the film, or '**neg**ative' otherwise.

> Maas, Andrew L., et al. [Learning Word Vectors for Sentiment Analysis](http://ai.stanford.edu/~amaas/data/sentiment/). In _Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies_. Association for Computational Linguistics, 2011.

We begin by using some Jupyter Notebook magic to download and extract the dataset.

In [1]:
%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

--2020-09-13 14:16:54--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘../data/aclImdb_v1.tar.gz’

../data/aclImdb_v1. 100%[===================>]  80.23M  25.6MB/s    in 3.6s    

2020-09-13 14:16:57 (22.4 MB/s) - ‘../data/aclImdb_v1.tar.gz’ saved [84125825/84125825]



## Step 2: Preparing the data

The data we have downloaded is split into various files, each of which contains a single review. It will be much easier going forward if we combine these individual files into two large files, one for training and one for testing.

In [9]:
import os
import glob

def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        # print("data[data_type] : ", data)
        # print("labels[data_type] : ", labels)
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            # print("data[data_type][sentiment] : ", data)
            # print("labels[data_type][sentiment] : ", labels)
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            print("path : ", path)
            
            
            files = glob.glob(path)
            # print("current data : ", data)
            # print("files : ", files)
            
            for f in files:
                # print("f : ", f)
                
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                # print("current data : ", data)
                # print("current labels : ", labels)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [8]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

path :  ../data/aclImdb/train/pos/*.txt
f :  ../data/aclImdb/train/pos/2054_9.txt
f :  ../data/aclImdb/train/pos/5629_10.txt
f :  ../data/aclImdb/train/pos/6191_9.txt
f :  ../data/aclImdb/train/pos/10872_7.txt
f :  ../data/aclImdb/train/pos/11490_9.txt
f :  ../data/aclImdb/train/pos/11969_10.txt
f :  ../data/aclImdb/train/pos/6992_10.txt
f :  ../data/aclImdb/train/pos/4109_10.txt
f :  ../data/aclImdb/train/pos/5556_10.txt
f :  ../data/aclImdb/train/pos/2451_8.txt
f :  ../data/aclImdb/train/pos/1650_10.txt
f :  ../data/aclImdb/train/pos/1_7.txt
f :  ../data/aclImdb/train/pos/2255_8.txt
f :  ../data/aclImdb/train/pos/414_10.txt
f :  ../data/aclImdb/train/pos/2746_10.txt
f :  ../data/aclImdb/train/pos/3003_9.txt
f :  ../data/aclImdb/train/pos/9611_8.txt
f :  ../data/aclImdb/train/pos/155_10.txt
f :  ../data/aclImdb/train/pos/12479_10.txt
f :  ../data/aclImdb/train/pos/11741_10.txt
f :  ../data/aclImdb/train/pos/12423_10.txt
f :  ../data/aclImdb/train/pos/2465_10.txt
f :  ../data/aclImdb/t

f :  ../data/aclImdb/train/pos/6615_8.txt
f :  ../data/aclImdb/train/pos/5188_8.txt
f :  ../data/aclImdb/train/pos/1307_10.txt
f :  ../data/aclImdb/train/pos/10957_10.txt
f :  ../data/aclImdb/train/pos/3217_8.txt
f :  ../data/aclImdb/train/pos/7553_10.txt
f :  ../data/aclImdb/train/pos/2903_9.txt
f :  ../data/aclImdb/train/pos/999_10.txt
f :  ../data/aclImdb/train/pos/2799_10.txt
f :  ../data/aclImdb/train/pos/4067_8.txt
f :  ../data/aclImdb/train/pos/6594_10.txt
f :  ../data/aclImdb/train/pos/4907_8.txt
f :  ../data/aclImdb/train/pos/1847_10.txt
f :  ../data/aclImdb/train/pos/4444_10.txt
f :  ../data/aclImdb/train/pos/9398_9.txt
f :  ../data/aclImdb/train/pos/9229_7.txt
f :  ../data/aclImdb/train/pos/2553_7.txt
f :  ../data/aclImdb/train/pos/10633_9.txt
f :  ../data/aclImdb/train/pos/2363_7.txt
f :  ../data/aclImdb/train/pos/2386_8.txt
f :  ../data/aclImdb/train/pos/589_10.txt
f :  ../data/aclImdb/train/pos/11839_9.txt
f :  ../data/aclImdb/train/pos/4428_10.txt
f :  ../data/aclImdb/tr

f :  ../data/aclImdb/train/pos/10541_10.txt
f :  ../data/aclImdb/train/pos/2258_7.txt
f :  ../data/aclImdb/train/pos/4961_10.txt
f :  ../data/aclImdb/train/pos/6026_8.txt
f :  ../data/aclImdb/train/pos/7322_10.txt
f :  ../data/aclImdb/train/pos/10108_10.txt
f :  ../data/aclImdb/train/pos/10585_9.txt
f :  ../data/aclImdb/train/pos/6061_10.txt
f :  ../data/aclImdb/train/pos/5917_7.txt
f :  ../data/aclImdb/train/pos/8495_8.txt
f :  ../data/aclImdb/train/pos/1921_9.txt
f :  ../data/aclImdb/train/pos/624_9.txt
f :  ../data/aclImdb/train/pos/10105_8.txt
f :  ../data/aclImdb/train/pos/11375_9.txt
f :  ../data/aclImdb/train/pos/2794_8.txt
f :  ../data/aclImdb/train/pos/9571_9.txt
f :  ../data/aclImdb/train/pos/5193_9.txt
f :  ../data/aclImdb/train/pos/11706_10.txt
f :  ../data/aclImdb/train/pos/11976_10.txt
f :  ../data/aclImdb/train/pos/8333_9.txt
f :  ../data/aclImdb/train/pos/1390_9.txt
f :  ../data/aclImdb/train/pos/2932_10.txt
f :  ../data/aclImdb/train/pos/3659_9.txt
f :  ../data/aclImdb

f :  ../data/aclImdb/train/pos/12098_8.txt
f :  ../data/aclImdb/train/pos/11029_7.txt
f :  ../data/aclImdb/train/pos/4692_10.txt
f :  ../data/aclImdb/train/pos/984_7.txt
f :  ../data/aclImdb/train/pos/9036_10.txt
f :  ../data/aclImdb/train/pos/10823_8.txt
f :  ../data/aclImdb/train/pos/3856_10.txt
f :  ../data/aclImdb/train/pos/6995_7.txt
f :  ../data/aclImdb/train/pos/1130_10.txt
f :  ../data/aclImdb/train/pos/2783_8.txt
f :  ../data/aclImdb/train/pos/8694_10.txt
f :  ../data/aclImdb/train/pos/3355_10.txt
f :  ../data/aclImdb/train/pos/4995_9.txt
f :  ../data/aclImdb/train/pos/2914_8.txt
f :  ../data/aclImdb/train/pos/6659_10.txt
f :  ../data/aclImdb/train/pos/3018_10.txt
f :  ../data/aclImdb/train/pos/106_10.txt
f :  ../data/aclImdb/train/pos/8589_7.txt
f :  ../data/aclImdb/train/pos/1558_10.txt
f :  ../data/aclImdb/train/pos/8371_10.txt
f :  ../data/aclImdb/train/pos/9506_8.txt
f :  ../data/aclImdb/train/pos/4368_10.txt
f :  ../data/aclImdb/train/pos/4240_9.txt
f :  ../data/aclImdb/

f :  ../data/aclImdb/train/pos/10571_8.txt
f :  ../data/aclImdb/train/pos/4749_8.txt
f :  ../data/aclImdb/train/pos/11114_10.txt
f :  ../data/aclImdb/train/pos/161_8.txt
f :  ../data/aclImdb/train/pos/9501_10.txt
f :  ../data/aclImdb/train/pos/8252_9.txt
f :  ../data/aclImdb/train/pos/7857_10.txt
f :  ../data/aclImdb/train/pos/9986_9.txt
f :  ../data/aclImdb/train/pos/11446_10.txt
f :  ../data/aclImdb/train/pos/5026_8.txt
f :  ../data/aclImdb/train/pos/10240_8.txt
f :  ../data/aclImdb/train/pos/1048_8.txt
f :  ../data/aclImdb/train/pos/6631_7.txt
f :  ../data/aclImdb/train/pos/5880_10.txt
f :  ../data/aclImdb/train/pos/11870_10.txt
f :  ../data/aclImdb/train/pos/1202_9.txt
f :  ../data/aclImdb/train/pos/5820_7.txt
f :  ../data/aclImdb/train/pos/998_7.txt
f :  ../data/aclImdb/train/pos/11848_10.txt
f :  ../data/aclImdb/train/pos/11269_8.txt
f :  ../data/aclImdb/train/pos/6202_7.txt
f :  ../data/aclImdb/train/pos/4450_9.txt
f :  ../data/aclImdb/train/pos/3176_9.txt
f :  ../data/aclImdb/t

...

f :  ../data/aclImdb/test/neg/712_3.txt
f :  ../data/aclImdb/test/neg/7755_3.txt
f :  ../data/aclImdb/test/neg/6774_4.txt
f :  ../data/aclImdb/test/neg/6007_1.txt
f :  ../data/aclImdb/test/neg/408_3.txt
f :  ../data/aclImdb/test/neg/6229_1.txt
f :  ../data/aclImdb/test/neg/9608_1.txt
f :  ../data/aclImdb/test/neg/8674_1.txt
f :  ../data/aclImdb/test/neg/5676_1.txt
f :  ../data/aclImdb/test/neg/10527_3.txt
f :  ../data/aclImdb/test/neg/8728_1.txt
f :  ../data/aclImdb/test/neg/3345_3.txt
f :  ../data/aclImdb/test/neg/6591_2.txt
f :  ../data/aclImdb/test/neg/2558_4.txt
f :  ../data/aclImdb/test/neg/8652_1.txt
f :  ../data/aclImdb/test/neg/7862_1.txt
f :  ../data/aclImdb/test/neg/8068_2.txt
f :  ../data/aclImdb/test/neg/4442_1.txt
f :  ../data/aclImdb/test/neg/9130_4.txt
f :  ../data/aclImdb/test/neg/6712_2.txt
f :  ../data/aclImdb/test/neg/540_1.txt
f :  ../data/aclImdb/test/neg/3585_2.txt
f :  ../data/aclImdb/test/neg/7766_1.txt
f :  ../data/aclImdb/test/neg/10102_2.txt
f :  ../data/aclI

f :  ../data/aclImdb/test/neg/4930_1.txt
f :  ../data/aclImdb/test/neg/12157_3.txt
f :  ../data/aclImdb/test/neg/12124_3.txt
f :  ../data/aclImdb/test/neg/8540_2.txt
f :  ../data/aclImdb/test/neg/5949_4.txt
f :  ../data/aclImdb/test/neg/7480_1.txt
f :  ../data/aclImdb/test/neg/3703_2.txt
f :  ../data/aclImdb/test/neg/1624_1.txt
f :  ../data/aclImdb/test/neg/4633_3.txt
f :  ../data/aclImdb/test/neg/3733_4.txt
f :  ../data/aclImdb/test/neg/8420_4.txt
f :  ../data/aclImdb/test/neg/2246_1.txt
f :  ../data/aclImdb/test/neg/7588_4.txt
f :  ../data/aclImdb/test/neg/8122_1.txt
f :  ../data/aclImdb/test/neg/11910_2.txt
f :  ../data/aclImdb/test/neg/6235_1.txt
f :  ../data/aclImdb/test/neg/11985_1.txt
f :  ../data/aclImdb/test/neg/2284_3.txt
f :  ../data/aclImdb/test/neg/1568_4.txt
f :  ../data/aclImdb/test/neg/12111_1.txt
f :  ../data/aclImdb/test/neg/12333_1.txt
f :  ../data/aclImdb/test/neg/1242_1.txt
f :  ../data/aclImdb/test/neg/447_4.txt
f :  ../data/aclImdb/test/neg/5182_3.txt
f :  ../dat

f :  ../data/aclImdb/test/neg/5637_2.txt
f :  ../data/aclImdb/test/neg/7245_1.txt
f :  ../data/aclImdb/test/neg/9828_4.txt
f :  ../data/aclImdb/test/neg/10548_1.txt
f :  ../data/aclImdb/test/neg/485_4.txt
f :  ../data/aclImdb/test/neg/7847_2.txt
f :  ../data/aclImdb/test/neg/3390_4.txt
f :  ../data/aclImdb/test/neg/1765_1.txt
f :  ../data/aclImdb/test/neg/5858_1.txt
f :  ../data/aclImdb/test/neg/5980_2.txt
f :  ../data/aclImdb/test/neg/11990_4.txt
f :  ../data/aclImdb/test/neg/7863_1.txt
f :  ../data/aclImdb/test/neg/9494_1.txt
f :  ../data/aclImdb/test/neg/1147_1.txt
f :  ../data/aclImdb/test/neg/2738_1.txt
f :  ../data/aclImdb/test/neg/9993_2.txt
f :  ../data/aclImdb/test/neg/4503_1.txt
f :  ../data/aclImdb/test/neg/7818_2.txt
f :  ../data/aclImdb/test/neg/4330_3.txt
f :  ../data/aclImdb/test/neg/12181_4.txt
f :  ../data/aclImdb/test/neg/5819_2.txt
f :  ../data/aclImdb/test/neg/10713_3.txt
f :  ../data/aclImdb/test/neg/882_2.txt
f :  ../data/aclImdb/test/neg/727_3.txt
f :  ../data/ac

f :  ../data/aclImdb/test/neg/2778_2.txt
f :  ../data/aclImdb/test/neg/8360_3.txt
f :  ../data/aclImdb/test/neg/337_1.txt
f :  ../data/aclImdb/test/neg/1702_2.txt
f :  ../data/aclImdb/test/neg/7968_4.txt
f :  ../data/aclImdb/test/neg/1115_3.txt
f :  ../data/aclImdb/test/neg/4261_1.txt
f :  ../data/aclImdb/test/neg/6782_3.txt
f :  ../data/aclImdb/test/neg/9721_4.txt
f :  ../data/aclImdb/test/neg/9804_1.txt
f :  ../data/aclImdb/test/neg/3859_1.txt
f :  ../data/aclImdb/test/neg/1222_2.txt
f :  ../data/aclImdb/test/neg/6132_1.txt
f :  ../data/aclImdb/test/neg/5083_4.txt
f :  ../data/aclImdb/test/neg/4179_1.txt
f :  ../data/aclImdb/test/neg/5106_1.txt
f :  ../data/aclImdb/test/neg/6811_4.txt
f :  ../data/aclImdb/test/neg/7834_3.txt
f :  ../data/aclImdb/test/neg/9791_2.txt
f :  ../data/aclImdb/test/neg/7470_4.txt
f :  ../data/aclImdb/test/neg/8917_4.txt
f :  ../data/aclImdb/test/neg/1849_4.txt
f :  ../data/aclImdb/test/neg/7155_2.txt
f :  ../data/aclImdb/test/neg/946_1.txt
f :  ../data/aclIm

f :  ../data/aclImdb/test/neg/4886_3.txt
f :  ../data/aclImdb/test/neg/6644_1.txt
f :  ../data/aclImdb/test/neg/7750_3.txt
f :  ../data/aclImdb/test/neg/6358_4.txt
f :  ../data/aclImdb/test/neg/7545_1.txt
f :  ../data/aclImdb/test/neg/10074_3.txt
f :  ../data/aclImdb/test/neg/11956_2.txt
f :  ../data/aclImdb/test/neg/3715_1.txt
f :  ../data/aclImdb/test/neg/11608_1.txt
f :  ../data/aclImdb/test/neg/7500_1.txt
f :  ../data/aclImdb/test/neg/10639_4.txt
f :  ../data/aclImdb/test/neg/5147_1.txt
f :  ../data/aclImdb/test/neg/9133_3.txt
f :  ../data/aclImdb/test/neg/10627_2.txt
f :  ../data/aclImdb/test/neg/7112_4.txt
f :  ../data/aclImdb/test/neg/1127_1.txt
f :  ../data/aclImdb/test/neg/8873_4.txt
f :  ../data/aclImdb/test/neg/5700_1.txt
f :  ../data/aclImdb/test/neg/6466_1.txt
f :  ../data/aclImdb/test/neg/3325_1.txt
f :  ../data/aclImdb/test/neg/2313_1.txt
f :  ../data/aclImdb/test/neg/10118_1.txt
f :  ../data/aclImdb/test/neg/10488_3.txt
f :  ../data/aclImdb/test/neg/10172_1.txt
f :  ../

In [29]:
labels

{'train': {'pos': [1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,


In [12]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [13]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

IMDb reviews (combined): train = 25000, test = 25000


In [15]:
train_y[100]

1

## Step 3: Processing the data

Now that we have our training and testing datasets merged and ready to use, we need to start processing the raw data into something that will be useable by our machine learning algorithm. To begin with, we remove any html formatting that may appear in the reviews and perform some standard natural language processing in order to homogenize the data.

In [16]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import *
stemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [47]:
stopwords.words("english")

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [70]:
import re
from bs4 import BeautifulSoup

def review_to_words(review):
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    print("words : ", words)
    
    return words

In [71]:
import pickle

cache_dir = os.path.join("../cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""
    
    
    # print("datatrain : ", data_train)
    # print("data_test : ", data_test)
    print("labels_train : ", labels_train)
    print("labels_test : ", labels_test)
    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        print(" if cache_file is not None: 1")
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            print("pass")
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        print("if cache_data is None:")
        
        # Preprocess training and test data to obtain words for each review
        #words_train = list(map(review_to_words, data_train))
        #words_test = list(map(review_to_words, data_test))
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        if cache_file is not None:
            print(" if cache_file is not None: 2")
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        print("cache data is exists")
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [72]:
total_train_X_data = len(train_X)
total_test_X_data = len(test_X)
total_train_Y_data = len(train_y)
total_train_Y_data = len(test_y)


print("total_train_X_data : ", total_train_X_data)
print("total_test_X_data : ", total_test_X_data)
print("total_train_Y_data : ", total_train_Y_data)
print("total_train_Y_data : ", total_train_Y_data)

total_train_X_data :  25000
total_test_X_data :  25000
total_train_Y_data :  25000
total_train_Y_data :  25000


In [76]:
array_target = 50

sample_train_X = train_X[:array_target]
sample_test_X = test_X[:array_target]
sample_train_y = train_y[:array_target]
sample_test_y = test_y[:array_target]

sample_train_X[0]

"Oliver Gruner is totally unknown to me. My friend showed me this film because he had seen Gruner in, what he called a pretty good sci-fi film, Nemesis. So as we watched this, we found ourselves fastforwarding through the BS drama parts just to get to the unbelievable action sequences. Gruner loves to kick and kick and kick. And kick! haha<br /><br />Gruner character is a graduate student who is forced to stay in a ghetto close to the one that he grew up in. He finds himself watching after the boy who lives with him because he really wants to join in the Mexican gang that keeps tormenting his family. Instead of joining up, Gruner tells the boy to fight back (against a gang? too crazy). Gruner plays a typical Van Damme character who kills everyone (or maims them pretty bad) and works to rid his block of these gangmembers.<br /><br />The plot was very cheesy and easy to think of. Gruner is probably not very well known because of his script-choosing if this movie is anything to compare po

In [77]:
result_sample_train_X, result_sample_test_X, result_sample_train_y, result_sample_test_y = preprocess_data(sample_train_X, sample_test_X, sample_train_y, sample_test_y)

labels_train :  [0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1]
labels_test :  [1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0]
 if cache_file is not None: 1
pass
if cache_data is None:
words :  ['oliv', 'gruner', 'total', 'unknown', 'friend', 'show', 'film', 'seen', 'gruner', 'call', 'pretti', 'good', 'sci', 'fi', 'film', 'nemesi', 'watch', 'found', 'fastforward', 'bs', 'drama', 'part', 'get', 'unbeliev', 'action', 'sequenc', 'gruner', 'love', 'kick', 'kick', 'kick', 'kick', 'hahagrun', 'charact', 'graduat', 'student', 'forc', 'stay', 'ghetto', 'close', 'one', 'grew', 'find', 'watch', 'boy', 'live', 'realli', 'want', 'join', 'mexican', 'gang', 'keep', 'torment', 'famili', 'instead', 'join', 'gruner', 'tell', 'boy', 'fight', 'back', 'gang', 'crazi', 'gruner', 'play', 'typic', 'van', '

words :  ['honesti', 'someon', 'told', 'director', 'lemoni', 'snicket', 'seri', 'unfortun', 'event', 'citi', 'angel', 'casper', 'go', 'neat', 'littl', 'low', 'budget', 'indi', 'film', 'real', 'good', 'say', 'person', 'must', 'joke', 'director', 'brad', 'siberl', 'realli', 'good', '10', 'item', 'less', 'similar', 'conceit', 'film', 'like', 'sunris', 'lost', 'translat', 'recent', 'involv', 'chanc', 'meet', 'two', 'peopl', 'serendip', 'put', 'probabl', 'never', 'cross', 'path', 'say', 'word', 'one', 'like', 'film', '10', 'item', 'less', 'focus', 'relationship', 'build', 'charact', 'come', 'understand', 'build', 'strength', 'weak', 'stori', 'involv', 'morgan', 'freeman', 'play', 'unnam', 'actor', 'goe', 'research', 'role', 'groceri', 'store', 'employe', 'upcom', 'independ', 'movi', 'thing', 'beyond', 'control', 'end', 'spend', 'day', 'ladi', '10', 'item', 'less', 'lane', 'play', 'paz', 'vega', 'rotten', 'marriag', 'hope', 'land', 'new', 'job', 'secretari', 'initi', 'freeman', 'charact', 'n

words :  ['film', 'artemisia', 'may', 'consid', 'treason', 'true', 'artist', 'licens', 'might', 'one', 'aver', 'document', 'histori', 'artemisia', 'gentileschi', 'subject', 'thumbscrew', 'still', 'affirm', 'r', 'ed', 'mari', 'garrard', 'gloria', 'steinem', 'eloqu', 'affirm', 'movi', 'differ', 'tortur', 'refus', 'condemn', 'lover', 'violat', 'may', 'movi', 'deviat', 'much', 'receiv', 'histori', 'yet', 'still', 'inform', 'human', 'heart', 'answer', 'hard', 'find', 'movi', 'director', 'cast', 'fill', 'gape', 'hole', 'histor', 'record', 'power', 'imagin', 'led', 'conclus', 'differ', 'record', 'find', 'record', 'movi', 'compel', 'movi', 'seem', 'histori', 'artemisia', 'painter', 'els', 'vision', 'frame', 'ravish', 'sic', 'film', 'composit', 'truli', 'grate', 'seldom', 'seen', 'movi', 'compel', 'eye', 'david', 'broadhurst']
words :  ['favorit', 'part', 'film', 'old', 'man', 'attempt', 'cure', 'neighbor', 'ill', 'put', 'strong', 'medicin', 'bath', 'sens', 'famili', 'sens', 'commun']
words :  

words :  ['movi', 'basic', 'human', 'relat', 'interact', 'main', 'charact', 'old', 'ladi', 'twilight', 'life', 'start', 'journey', 'past', 'analysi', 'live', 'life', 'journey', 'precipit', 'son', 'econom', 'crisi', 'intent', 'put', 'nurs', 'home', 'honest', 'look', 'issu', 'ask', 'point', 'life', 'plenti', 'secondari', 'idea', 'discuss', 'movi', 'famili', 'legaci', 'real', 'love', 'marriag', 'destini', 'although', 'type', 'movi', 'melodrama', 'noth', 'new', 'one', 'use', 'watch', 'famili', 'member', 'discuss', 'idea', 'good', 'perform', 'actor', 'charact', 'believ', 'time', 'charact', 'mayb', 'fulli', 'develop', 'realli', 'recommend', 'movi', 'quiet', 'saturday', 'afternoon']
words :  ['rip', '1984', 'hit', 'gremlin', 'quit', 'possibl', 'biggest', 'train', 'wreck', 'movi', 'ever', 'made', 'even', 'b', 'grade', 'movi', 'cheap', 'horror', 'movi', 'platform', 'complet', 'dwarf', 'movi', 'term', 'plot', 'act', 'good', 'begin', 'random', 'old', 'secur', 'guard', 'younger', 'punki', 'secur',

words :  ['decid', 'whether', 'one', 'favourit', 'movi', 'good', 'thriller', 'emot', 'core', 'still', 'decid', 'definit', 'like', 'first', 'movi', 'terri', 'gilliam', 'seen', 'first', 'impress', 'engag', 'till', 'end', 'complex', 'confus', 'movi', 'set', 'futur', 'man', 'jame', 'cole', 'bruce', 'willi', 'sent', 'futur', 'order', 'get', 'inform', 'past', '1996', 'specif', 'viru', 'kill', '5', 'billion', 'peopl', 'sent', 'futur', 'get', 'inform', 'also', 'involv', 'psychiatrist', 'call', 'kathryn', 'railli', 'love', 'stori', 'portray', 'beauti', 'realli', 'feel', 'long', 'love', 'long', 'regular', 'life', 'loos', 'end', 'tie', 'interest', 'manner', 'end', 'one', 'thing', 'like', 'movi', 'unlik', 'post', 'apocalypt', 'movi', 'movi', 'prefer', 'give', 'bore', 'social', 'commentari', 'instead', 'focus', 'one', 'guy', 'long', 'regular', 'life', 'want', 'see', 'ocean', 'especi', 'poignant', 'line', 'movi', 'choos', 'focu', 'tension', 'confus', 'person', 'mind', 'therefor', 'exactli', 'post', 

words :  ['scifi', 'past', 'weekend', 'check', 'scienc', 'fiction', 'vampir', 'erika', 'eleniak', 'could', 'go', 'wrong', 'b', 'movi', 'lot', 'start', 'even', 'classifi', 'b', 'movi', 'would', 'put', 'leagu', 'roger', 'corman', 'movi', 'even', 'meet', 'expect', 'money', 'spent', 'contact', 'lens', 'vampir', 'secondli', 'cast', 'horribl', 'ye', 'cast', 'udo', 'kier', 'captain', 'demet', 'smart', 'move', 'director', 'clearli', 'even', 'get', 'kier', 'memor', 'line', 'cast', 'eleniak', 'vampir', 'movi', 'also', 'smart', 'move', 'mean', 'bunch', 'horni', 'guy', 'go', 'buy', 'rent', 'record', 'flick', 'watch', 'get', 'seduc', 'vampir', 'director', 'writer', 'produc', 'screw', 'one', 'grant', 'got', 'money', 'poor', 'unfortu', 'soul', 'enjoy', 'watch', 'vampir', 'movi', 'hot', 'women', 'one', 'go', 'rememb', 'movi', 'anoth', 'two', 'three', 'year', 'thirdli', 'littl', 'thing', 'emphas', 'lazi', 'movi', 'exampl', 'van', 'hels', 'call', 'cross', 'crucifix', 'mina', 'stake', 'coffin', 'viewer',

words :  ['alway', 'fan', 'jackass', 'well', 'viva', 'la', 'bam', 'wildboyz', 'fan', 'someth', 'expect', 'high', 'whatev', 'hero', 'might', 'star', 'one', 'thing', 'learn', 'expect', 'lot', 'peopl', 'simpli', 'love', 'watch', 'listen', 'never', 'expect', 'much', 'caus', '99', '100', 'time', 'get', 'disappoint', 'although', 'heard', 'jackass', '2', 'come', 'thought', 'even', 'turn', 'expect', 'movi', 'result', 'sat', 'today', 'readi', 'laugh', 'also', 'readi', 'say', 'end', 'well', 'ok', 'littl', 'disappoint', 'wrong', 'everi', 'singl', 'member', 'jackass', 'crew', 'bring', 'movi', 'way', 'first', 'one', 'show', 'one', 'crazi', 'ass', 'stunt', 'make', 'whole', 'world', 'see', 'noth', 'wont', 'tri', 'harm', 'love', 'cri', 'eye', 'laugh', 'first', 'minut', 'till', 'last', 'second', 'movi', 'time', 'even', 'shout', 'laughter', 'abl', 'control', 'stunt', 'stunt', 'prank', 'prank', 'hilari', 'comment', 'flow', 'simpli', 'get', 'better', 'amaz', 'start', 'till', 'end', 'guarante', 'make', 'la

words :  ['georg', 'sander', 'play', 'saint', 'penultim', 'time', 'good', 'job', 'good', 'script', 'usual', 'good', 'rko', 'cast', 'around', 'non', 'charteri', 'stori', 'bristl', 'murder', 'good', 'clean', 'fun', 'thread', '1', 'new', 'york', 'polic', 'inspector', 'fernack', 'templar', 'friend', 'frame', 'corrupt', 'scandal', 'disgrac', 'st', 'come', 'london', 'tri', 'put', 'thing', 'right', 'nice', 'simpl', 'far', 'realli', '90', '000', 'world', 'thread', '2', 'anoth', 'tale', 'woman', 'take', 'reveng', 'peopl', 'murder', 'brother', 'wendi', 'barri', 'well', 'bump', 'nasti', 'men', 'saint', 'fall', 'love', 'boot', 'includ', 'baddi', 'direct', 'protect', 'fernack', 'cellar', 'creepi', 'shot', 'dead', 'stare', 'car', 'take', 'back', 'got', 'paul', 'guilfoyl', 'pearli', 'gate', 'must', 'suppos', 'homosexu', 'wit', 'dress', 'gown', 'begin', 'later', 'beguil', 'comment', 'st', 'think', 'keep', 'pet', 'palm', 'spring', 'fernack', 'play', 'jonathan', 'hale', 'usual', 'time', 'beaten', 'defla

words :  ['5', 'year', 'old', 'daughter', 'barbi', 'seri', 'movi', 'mix', 'feel', 'want', 'buy', 'whole', 'barbi', 'doll', 'imag', 'thing', 'recogn', 'movi', 'market', 'ploy', 'convinc', 'young', 'girl', 'buy', 'doll', 'make', 'money', 'mattel', 'morn', 'though', 'ask', 'watch', 'movi', 'lazi', 'saturday', 'morn', 'much', 'els', 'agre', 'know', 'movi', 'made', 'help', 'market', 'doll', 'seem', 'lose', 'appeal', 'bit', 'heard', 'doll', 'market', 'movi', 'like', 'bit', 'whichev', 'case', 'admit', 'somewhat', 'surpris', 'half', 'bad', 'fun', 'imagin', 'stori', 'full', 'magic', 'place', 'peopl', 'memor', 'charact', 'good', 'evil', 'essenti', 'annika', 'play', 'barbi', 'find', 'way', 'build', 'wand', 'light', 'revers', 'evil', 'spell', 'wizard', 'wenlock', 'among', 'thing', 'turn', 'sister', 'fli', 'hors', 'parent', 'stone', 'anim', 'pretti', 'good', 'disney', 'calibr', 'one', 'think', 'disney', 'standard', 'aspir', 'gener', 'pretti', 'good', 'movi', 'obvious', 'tailor', 'young', 'girl', 'r

In [ ]:
result_sample_train_X

In [62]:
cache_file="preprocessed_data.pkl"
with open(os.path.join(cache_dir, cache_file), "rb") as f:
            cache_data = pickle.load(f)

In [63]:
cache_data

{'words_train': [['oliv',
   'gruner',
   'total',
   'unknown',
   'friend',
   'show',
   'film',
   'seen',
   'gruner',
   'call',
   'pretti',
   'good',
   'sci',
   'fi',
   'film',
   'nemesi',
   'watch',
   'found',
   'fastforward',
   'bs',
   'drama',
   'part',
   'get',
   'unbeliev',
   'action',
   'sequenc',
   'gruner',
   'love',
   'kick',
   'kick',
   'kick',
   'kick',
   'hahagrun',
   'charact',
   'graduat',
   'student',
   'forc',
   'stay',
   'ghetto',
   'close',
   'one',
   'grew',
   'find',
   'watch',
   'boy',
   'live',
   'realli',
   'want',
   'join',
   'mexican',
   'gang',
   'keep',
   'torment',
   'famili',
   'instead',
   'join',
   'gruner',
   'tell',
   'boy',
   'fight',
   'back',
   'gang',
   'crazi',
   'gruner',
   'play',
   'typic',
   'van',
   'damm',
   'charact',
   'kill',
   'everyon',
   'maim',
   'pretti',
   'bad',
   'work',
   'rid',
   'block',
   'gangmemb',
   'plot',
   'cheesi',
   'easi',
   'think',
   'gru

In [73]:
# Preprocess data
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

labels_train :  [0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,

words :  ['haha', 'smile', 'smile', 'actual', 'made', 'way', 'movi', 'like', 'say', 'someth', 'guess', 'movi', 'creat', 'think', 'sort', 'psycholog', 'test', 'like', 'sort', 'drug', 'take', 'place', 'never', 'wittgenstein', 'wrote', 'famou', 'first', 'philosoph', 'piec', 'tractacu', 'sp', 'said', 'meaningless', 'useless', 'read', 'done', 'would', 'take', 'new', 'level', 'like', 'ladder', 'could', 'throw', 'away', 'work', 'see', 'thing', 'clariti', 'true', 'understand', 'movi', 'think', 'movi', 'without', 'doubt', 'worst', 'movi', 'seen', 'long', 'long', 'time', 'uniqu', 'way', 'first', 'snipe', 'love', 'watch', 'guy', 'kick', 'ass', 'variou', 'movi', 'suffer', 'weak', 'one', 'howev', 'although', 'know', 'movi', 'might', 'suck', 'would', 'never', 'suspect', 'could', 'bad', 'actual', 'fun', 'mean', 'snipe', 'know', 'might', 'good', 'alright', 'right', 'smile', 'thing', 'everi', 'level', 'pure', 'boredom', 'pure', 'unorigin', 'refer', 'profession', 'dead', 'obviou', 'yet', 'poorli', 'done

words :  ['honesti', 'someon', 'told', 'director', 'lemoni', 'snicket', 'seri', 'unfortun', 'event', 'citi', 'angel', 'casper', 'go', 'neat', 'littl', 'low', 'budget', 'indi', 'film', 'real', 'good', 'say', 'person', 'must', 'joke', 'director', 'brad', 'siberl', 'realli', 'good', '10', 'item', 'less', 'similar', 'conceit', 'film', 'like', 'sunris', 'lost', 'translat', 'recent', 'involv', 'chanc', 'meet', 'two', 'peopl', 'serendip', 'put', 'probabl', 'never', 'cross', 'path', 'say', 'word', 'one', 'like', 'film', '10', 'item', 'less', 'focus', 'relationship', 'build', 'charact', 'come', 'understand', 'build', 'strength', 'weak', 'stori', 'involv', 'morgan', 'freeman', 'play', 'unnam', 'actor', 'goe', 'research', 'role', 'groceri', 'store', 'employe', 'upcom', 'independ', 'movi', 'thing', 'beyond', 'control', 'end', 'spend', 'day', 'ladi', '10', 'item', 'less', 'lane', 'play', 'paz', 'vega', 'rotten', 'marriag', 'hope', 'land', 'new', 'job', 'secretari', 'initi', 'freeman', 'charact', 'n

words :  ['film', 'artemisia', 'may', 'consid', 'treason', 'true', 'artist', 'licens', 'might', 'one', 'aver', 'document', 'histori', 'artemisia', 'gentileschi', 'subject', 'thumbscrew', 'still', 'affirm', 'r', 'ed', 'mari', 'garrard', 'gloria', 'steinem', 'eloqu', 'affirm', 'movi', 'differ', 'tortur', 'refus', 'condemn', 'lover', 'violat', 'may', 'movi', 'deviat', 'much', 'receiv', 'histori', 'yet', 'still', 'inform', 'human', 'heart', 'answer', 'hard', 'find', 'movi', 'director', 'cast', 'fill', 'gape', 'hole', 'histor', 'record', 'power', 'imagin', 'led', 'conclus', 'differ', 'record', 'find', 'record', 'movi', 'compel', 'movi', 'seem', 'histori', 'artemisia', 'painter', 'els', 'vision', 'frame', 'ravish', 'sic', 'film', 'composit', 'truli', 'grate', 'seldom', 'seen', 'movi', 'compel', 'eye', 'david', 'broadhurst']
words :  ['favorit', 'part', 'film', 'old', 'man', 'attempt', 'cure', 'neighbor', 'ill', 'put', 'strong', 'medicin', 'bath', 'sens', 'famili', 'sens', 'commun']
words :  

words :  ['movi', 'basic', 'human', 'relat', 'interact', 'main', 'charact', 'old', 'ladi', 'twilight', 'life', 'start', 'journey', 'past', 'analysi', 'live', 'life', 'journey', 'precipit', 'son', 'econom', 'crisi', 'intent', 'put', 'nurs', 'home', 'honest', 'look', 'issu', 'ask', 'point', 'life', 'plenti', 'secondari', 'idea', 'discuss', 'movi', 'famili', 'legaci', 'real', 'love', 'marriag', 'destini', 'although', 'type', 'movi', 'melodrama', 'noth', 'new', 'one', 'use', 'watch', 'famili', 'member', 'discuss', 'idea', 'good', 'perform', 'actor', 'charact', 'believ', 'time', 'charact', 'mayb', 'fulli', 'develop', 'realli', 'recommend', 'movi', 'quiet', 'saturday', 'afternoon']
words :  ['rip', '1984', 'hit', 'gremlin', 'quit', 'possibl', 'biggest', 'train', 'wreck', 'movi', 'ever', 'made', 'even', 'b', 'grade', 'movi', 'cheap', 'horror', 'movi', 'platform', 'complet', 'dwarf', 'movi', 'term', 'plot', 'act', 'good', 'begin', 'random', 'old', 'secur', 'guard', 'younger', 'punki', 'secur',

...

words :  ['titl', 'robot', 'jox', '1990', 'director', 'stuart', 'gordon', 'cast', 'gari', 'graham', 'ann', 'mari', 'johnson', 'paul', 'koslo', 'review', 'stuart', 'gordon', 'usual', 'associ', 'extrem', 'gori', 'horror', 'film', 'anim', 'beyond', 'dagon', 'castl', 'freak', 'took', 'small', 'detour', 'littl', 'sci', 'fi', 'flick', 'stress', 'word', 'littl', 'sinc', 'low', 'budget', 'flick', 'lie', 'main', 'weak', 'stori', 'take', 'place', 'futur', 'world', 'great', 'superpow', 'accord', 'movi', 'unit', 'state', 'russia', 'duke', 'differ', 'go', 'full', 'blown', 'world', 'war', 'fight', 'gladiat', 'style', 'battl', 'gigant', 'robot', 'hero', 'achil', 'must', 'go', 'evil', 'russian', 'robot', 'fighter', 'call', 'alexand', 'lot', 'cheap', 'stop', 'motion', 'anim', 'ensu', 'well', 'idea', 'awesom', 'guess', 'great', 'nation', 'settlel', 'territori', 'disput', 'giant', 'robot', 'interest', 'premis', 'one', 'could', 'handl', 'properli', 'proper', 'budget', 'avail', 'unfortun', 'could', 'fun', 

words :  ['old', 'secur', 'guard', 'guy', 'die', 'kevin', 'world', 'biggest', 'wuss', 'kevin', 'want', 'impress', 'incred', 'insensit', 'bratti', 'virgin', 'girlfriend', 'ami', 'return', 'work', 'random', 'hous', 'find', 'friend', 'sexual', 'confus', 'red', 'short', 'kyle', 'truli', 'revolt', 'sluttish', 'daphn', 'soon', 'join', 'daphn', 'boyfriend', 'trigger', 'happi', 'sex', 'craze', 'macho', 'lunkhead', 'nick', 'titl', 'creatur', 'horrid', 'littl', 'dogear', 'puppet', 'kill', 'peopl', 'give', 'heart', 'desir', 'kyle', 'heart', 'desir', 'mate', 'creepi', 'yucki', 'woman', 'spandex', 'nick', 'heart', 'desir', 'throw', 'grenad', 'grade', 'school', 'cafeteria', 'mean', 'nightclub', 'kevin', 'heart', 'desir', 'beat', 'skinni', 'thug', 'nunchuck', 'ami', 'heart', 'desir', 'disgust', 'slut', 'daphn', 'alreadi', 'disgust', 'slut', 'heart', 'desir', 'along', 'way', 'truli', 'hideou', 'band', 'sing', 'truli', 'odd', 'song', 'hobgoblin', 'randomli', 'go', 'back', 'came', 'blow', 'citizen', 'ka

words :  ['could', 'seen', 'coup', 'toward', 'sexual', 'revolut', 'purpos', 'use', 'quotat', 'word', 'jean', 'eustach', 'wrote', 'direct', 'mother', 'whore', 'poetic', 'damn', 'critiqu', 'seem', 'get', 'enough', 'love', 'messag', 'film', 'hope', 'messag', 'would', 'come', 'fact', 'els', 'ben', 'hur', 'length', 'featur', 'offer', 'order', 'love', 'honestli', 'level', 'happi', 'real', 'truth', 'possibl', 'two', 'lover', 'tri', 'outcom', 'one', 'realli', 'realli', 'want', 'feel', 'even', 'express', 'say', 'want', 'truth', 'relationship', 'alexandr', 'jean', 'pierr', 'leaud', 'women', 'around', 'twenti', 'someth', 'pseudo', 'intellectu', 'seem', 'job', 'live', 'woman', 'mari', 'bernadett', 'lafont', 'slightli', 'older', 'usual', 'alway', 'lover', 'last', 'possibl', 'love', 'life', 'left', 'right', 'away', 'pick', 'woman', 'see', 'street', 'veronika', 'fran', 'ois', 'lebrun', 'perhap', 'remind', 'soon', 'unfold', 'subtli', 'torrid', 'love', 'triangl', 'ever', 'put', 'film', 'psycholog', 'st

words :  ['comment', 'miniseri', 'perspect', 'someon', 'read', 'novel', 'first', 'perspect', 'honestli', 'say', 'enjoy', 'see', 'rebroadcast', 'anytim', 'recent', 'specif', 'mini', 'seriou', 'problem', '1', 'terribl', 'miscast', 'actor', 'play', 'younger', 'gener', '15', '20', 'year', 'older', 'charact', 'ali', 'mcgraw', '45', 'time', 'play', 'natali', 'jastrow', 'suppos', '26', 'jan', 'michael', 'vincent', '39', 'time', 'play', 'byron', 'henri', 'suppos', '22', 'henri', 'children', 'pamela', 'tudsburi', 'also', 'play', 'actor', 'way', 'old', 'charact', 'suppos', '20', '2', 'act', 'absolut', 'aw', 'ali', 'mcgraw', 'time', 'almost', 'made', 'mini', 'unwatch', 'seen', 'convinc', 'perform', 'high', 'school', 'play', '3', 'direct', 'poor', 'fair', 'ali', 'mcgraw', 'bad', 'act', 'charact', 'develop', 'probabl', 'direct', 'portray', 'hitler', 'way', 'overdon', 'charact', 'came', 'look', 'behav', 'like', 'cartoon', 'villain', 'charismat', 'sometim', 'charm', 'alway', 'diabol', 'geniu', 'herma

KeyboardInterrupt: 

### Extract Bag-of-Words features

For the model we will be implementing, rather than using the reviews directly, we are going to transform each review into a Bag-of-Words feature representation. Keep in mind that 'in the wild' we will only have access to the training set so our transformer can only use the training set to construct a representation.

In [78]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.externals import joblib
import joblib
# joblib is an enhanced version of pickle that is more efficient for storing NumPy arrays

def extract_BoW_features(words_train, words_test, vocabulary_size=5000,
                         cache_dir=cache_dir, cache_file="bow_features.pkl"):
    """Extract Bag-of-Words for a given set of documents, already preprocessed into words."""
    
    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = joblib.load(f)
            print("Read features from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Fit a vectorizer to training documents and use it to transform them
        # NOTE: Training documents have already been preprocessed and tokenized into words;
        #       pass in dummy functions to skip those steps, e.g. preprocessor=lambda x: x
        vectorizer = CountVectorizer(max_features=vocabulary_size,
                preprocessor=lambda x: x, tokenizer=lambda x: x)  # already preprocessed
        features_train = vectorizer.fit_transform(words_train).toarray()

        # Apply the same vectorizer to transform the test documents (ignore unknown words)
        features_test = vectorizer.transform(words_test).toarray()
        
        # NOTE: Remember to convert the features using .toarray() for a compact representation
        
        # Write to cache file for future runs (store vocabulary as well)
        if cache_file is not None:
            vocabulary = vectorizer.vocabulary_
            cache_data = dict(features_train=features_train, features_test=features_test,
                             vocabulary=vocabulary)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                joblib.dump(cache_data, f)
            print("Wrote features to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        features_train, features_test, vocabulary = (cache_data['features_train'],
                cache_data['features_test'], cache_data['vocabulary'])
    
    # Return both the extracted features as well as the vocabulary
    return features_train, features_test, vocabulary

In [79]:
train_X, test_X, vocabulary = extract_BoW_features(result_sample_train_X, result_sample_test_X)

Wrote features to cache file: bow_features.pkl


In [87]:
train_X

,0,1,2,3,4,5,6,7,8,9,...,2107,2108,2109,2110,2111,2112,2113,2114,2115,2116


In [82]:
vocabulary

{'oliv': 1291,
 'gruner': 838,
 'total': 1917,
 'unknown': 1974,
 'friend': 759,
 'show': 1668,
 'film': 703,
 'seen': 1624,
 'call': 287,
 'pretti': 1415,
 'good': 812,
 'sci': 1600,
 'fi': 696,
 'nemesi': 1246,
 'watch': 2035,
 'found': 744,
 'fastforward': 682,
 'bs': 272,
 'drama': 564,
 'part': 1328,
 'get': 790,
 'unbeliev': 1956,
 'action': 49,
 'sequenc': 1639,
 'love': 1102,
 'kick': 1016,
 'hahagrun': 849,
 'charact': 327,
 'graduat': 821,
 'student': 1803,
 'forc': 734,
 'stay': 1766,
 'ghetto': 791,
 'close': 376,
 'one': 1293,
 'grew': 830,
 'find': 708,
 'boy': 250,
 'live': 1090,
 'realli': 1477,
 'want': 2025,
 'join': 994,
 'mexican': 1173,
 'gang': 772,
 'keep': 1009,
 'torment': 1915,
 'famili': 670,
 'instead': 966,
 'tell': 1867,
 'fight': 698,
 'back': 166,
 'crazi': 452,
 'play': 1376,
 'typic': 1950,
 'van': 1992,
 'damm': 479,
 'kill': 1018,
 'everyon': 638,
 'maim': 1118,
 'bad': 168,
 'work': 2082,
 'rid': 1528,
 'block': 224,
 'gangmemb': 773,
 'plot': 1380,

In [86]:
train_X

,0,1,2,3,4,5,6,7,8,9,...,2107,2108,2109,2110,2111,2112,2113,2114,2115,2116


In [ ]:
# Extract Bag of Words features for both training and test datasets
# train_X, test_X, vocabulary = extract_BoW_features(train_X, test_X)

## Step 4: Classification using XGBoost

Now that we have created the feature representation of our training (and testing) data, it is time to start setting up and using the XGBoost classifier provided by SageMaker.

### (TODO) Writing the dataset

The XGBoost classifier that we will be using requires the dataset to be written to a file and stored using Amazon S3. To do this, we will start by splitting the training dataset into two parts, the data we will train the model with and a validation set. Then, we will write those datasets to a file and upload the files to S3. In addition, we will write the test set input to a file and upload the file to S3. This is so that we can use SageMakers Batch Transform functionality to test our model once we've fit it.

In [83]:
import pandas as pd

val_X = pd.DataFrame(train_X[:10000])
train_X = pd.DataFrame(train_X[10000:])

val_y = pd.DataFrame(train_y[:10000])
train_y = pd.DataFrame(train_y[10000:])
# TODO: Split the train_X and train_y arrays into the DataFrames val_X, train_X and val_y, train_y. Make sure that
#       val_X and val_y contain 10 000 entires while train_X and train_y contain the remaining 15 000 entries.


In [85]:
val_X

,0,1,2,3,4,5,6,7,8,9,...,2107,2108,2109,2110,2111,2112,2113,2114,2115,2116
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,2,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


The documentation for the XGBoost algorithm in SageMaker requires that the saved datasets should contain no headers or index and that for the training and validation data, the label should occur first for each sample.

For more information about this and other algorithms, the SageMaker developer documentation can be found on __[Amazon's website.](https://docs.aws.amazon.com/sagemaker/latest/dg/)__

In [ ]:
# First we make sure that the local directory in which we'd like to store the training and validation csv files exists.
data_dir = '../data/xgboost'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [ ]:
# First, save the test data to test.csv in the data_dir directory. Note that we do not save the associated ground truth
# labels, instead we will use them later to compare with our model output.

pd.DataFrame(test_X).to_csv(os.path.join(data_dir, 'test.csv'), header=False, index=False)

# TODO: Save the training and validation data to train.csv and validation.csv in the data_dir directory.
#       Make sure that the files you create are in the correct format.


In [ ]:
# To save a bit of memory we can set text_X, train_X, val_X, train_y and val_y to None.

test_X = train_X = val_X = train_y = val_y = None

### (TODO) Uploading Training / Validation files to S3

Amazon's S3 service allows us to store files that can be access by both the built-in training models such as the XGBoost model we will be using as well as custom models such as the one we will see a little later.

For this, and most other tasks we will be doing using SageMaker, there are two methods we could use. The first is to use the low level functionality of SageMaker which requires knowing each of the objects involved in the SageMaker environment. The second is to use the high level functionality in which certain choices have been made on the user's behalf. The low level approach benefits from allowing the user a great deal of flexibility while the high level approach makes development much quicker. For our purposes we will opt to use the high level approach although using the low-level approach is certainly an option.

Recall the method `upload_data()` which is a member of object representing our current SageMaker session. What this method does is upload the data to the default bucket (which is created if it does not exist) into the path described by the key_prefix variable. To see this for yourself, once you have uploaded the data files, go to the S3 console and look to see where the files have been uploaded.

For additional resources, see the __[SageMaker API documentation](http://sagemaker.readthedocs.io/en/latest/)__ and in addition the __[SageMaker Developer Guide.](https://docs.aws.amazon.com/sagemaker/latest/dg/)__

In [ ]:
import sagemaker

session = sagemaker.Session() # Store the current SageMaker session

# S3 prefix (which folder will we use)
prefix = 'sentiment-xgboost'

# TODO: Upload the test.csv, train.csv and validation.csv files which are contained in data_dir to S3 using sess.upload_data().
test_location = None
val_location = None
train_location = None

### (TODO) Creating the XGBoost model

Now that the data has been uploaded it is time to create the XGBoost model. To begin with, we need to do some setup. At this point it is worth discussing what a model is in SageMaker. It is easiest to think of a model of comprising three different objects in the SageMaker ecosystem, which interact with one another.

- Model Artifacts
- Training Code (Container)
- Inference Code (Container)

The Model Artifacts are what you might think of as the actual model itself. For example, if you were building a neural network, the model artifacts would be the weights of the various layers. In our case, for an XGBoost model, the artifacts are the actual trees that are created during training.

The other two objects, the training code and the inference code are then used the manipulate the training artifacts. More precisely, the training code uses the training data that is provided and creates the model artifacts, while the inference code uses the model artifacts to make predictions on new data.

The way that SageMaker runs the training and inference code is by making use of Docker containers. For now, think of a container as being a way of packaging code up so that dependencies aren't an issue.

In [ ]:
from sagemaker import get_execution_role

# Our current execution role is require when creating the model as the training
# and inference code will need to access the model artifacts.
role = get_execution_role()

In [ ]:
# We need to retrieve the location of the container which is provided by Amazon for using XGBoost.
# As a matter of convenience, the training and inference code both use the same container.
from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(session.boto_region_name, 'xgboost')

In [ ]:
# TODO: Create a SageMaker estimator using the container location determined in the previous cell.
#       It is recommended that you use a single training instance of type ml.m4.xlarge. It is also
#       recommended that you use 's3://{}/{}/output'.format(session.default_bucket(), prefix) as the
#       output path.

xgb = None


# TODO: Set the XGBoost hyperparameters in the xgb object. Don't forget that in this case we have a binary
#       label so we should be using the 'binary:logistic' objective.


### Fit the XGBoost model

Now that our model has been set up we simply need to attach the training and validation datasets and then ask SageMaker to set up the computation.

In [ ]:
s3_input_train = sagemaker.s3_input(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data=val_location, content_type='csv')

In [ ]:
xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

### (TODO) Testing the model

Now that we've fit our XGBoost model, it's time to see how well it performs. To do this we will use SageMakers Batch Transform functionality. Batch Transform is a convenient way to perform inference on a large dataset in a way that is not realtime. That is, we don't necessarily need to use our model's results immediately and instead we can peform inference on a large number of samples. An example of this in industry might be peforming an end of month report. This method of inference can also be useful to us as it means to can perform inference on our entire test set. 

To perform a Batch Transformation we need to first create a transformer objects from our trained estimator object.

In [ ]:
# TODO: Create a transformer object from the trained model. Using an instance count of 1 and an instance type of ml.m4.xlarge
#       should be more than enough.
xgb_transformer = None

Next we actually perform the transform job. When doing so we need to make sure to specify the type of data we are sending so that it is serialized correctly in the background. In our case we are providing our model with csv data so we specify `text/csv`. Also, if the test data that we have provided is too large to process all at once then we need to specify how the data file should be split up. Since each line is a single entry in our data set we tell SageMaker that it can split the input on each line.

In [ ]:
# TODO: Start the transform job. Make sure to specify the content type and the split type of the test data.


Currently the transform job is running but it is doing so in the background. Since we wish to wait until the transform job is done and we would like a bit of feedback we can run the `wait()` method.

In [ ]:
xgb_transformer.wait()

Now the transform job has executed and the result, the estimated sentiment of each review, has been saved on S3. Since we would rather work on this file locally we can perform a bit of notebook magic to copy the file to the `data_dir`.

In [ ]:
!aws s3 cp --recursive $xgb_transformer.output_path $data_dir

The last step is now to read in the output from our model, convert the output to something a little more usable, in this case we want the sentiment to be either `1` (positive) or `0` (negative), and then compare to the ground truth labels.

In [ ]:
predictions = pd.read_csv(os.path.join(data_dir, 'test.csv.out'), header=None)
predictions = [round(num) for num in predictions.squeeze().values]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

## Optional: Clean up

The default notebook instance on SageMaker doesn't have a lot of excess disk space available. As you continue to complete and execute notebooks you will eventually fill up this disk space, leading to errors which can be difficult to diagnose. Once you are completely finished using a notebook it is a good idea to remove the files that you created along the way. Of course, you can do this from the terminal or from the notebook hub if you would like. The cell below contains some commands to clean up the created files from within the notebook.

In [ ]:
# First we will remove all of the files contained in the data_dir directory
!rm $data_dir/*

# And then we delete the directory itself
!rmdir $data_dir

# Similarly we will remove the files in the cache_dir directory and the directory itself
!rm $cache_dir/*
!rmdir $cache_dir